In [ ]:
import torch, torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
import numpy as np
import shap

batch_size = 128
num_epochs = 2
device = torch.device('mps')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout(),
            nn.MaxPool2d(2),
            nn.ReLU(),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        x = self.fc_layers(x)
        return x

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output.log(), target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output.log(), target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=False, transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.5)

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


model = model.to(torch.device('cpu'))

# MODEL IS TRAINED, NOW WE DO SOME SHAP

# since shuffle=True, this is a random sample of test data
batch = next(iter(test_loader))
images, labels = batch

background = images[:100]

imgs = []
for label in [0,1,2,3,4,5,6,7,8,9]:
  ind =  labels.tolist().index(label) 
  # plt.imshow( images[ind][0] )
  imgs.append( images[ind] )

test_images = torch.stack( imgs )

e = shap.DeepExplainer(model, background)
shap_values = e.shap_values(test_images)

shap_numpy = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values]
test_numpy = np.swapaxes(np.swapaxes(test_images.numpy(), 1, -1), 1, 2)

# plot the feature attributions
shap.image_plot(shap_numpy, -test_numpy)

In [27]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout(),
            nn.MaxPool2d(2),
            nn.ReLU(),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        x = self.fc_layers(x)
        return x


# pull the fashion MNIST datset from pythorch repos

In [28]:
import sys
sys.path.append('./../')

import torchvision
import torch
from pythorch_code import load_fashion_MNIST
import matplotlib.pyplot as plt


train_loader, test_loader = load_fashion_MNIST(batch_size=256)
device = torch.device('mps')

model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.02, momentum=0.5)

model.train()
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = torch.nn.functional.nll_loss(output.log(), target)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print(f'Train Epoch: {epoch} \tLoss: {loss.item()}')

Train Epoch: 0 	Loss: 2.3114898204803467
Train Epoch: 0 	Loss: 1.4362776279449463
Train Epoch: 0 	Loss: 1.0638426542282104
Train Epoch: 1 	Loss: 1.1164062023162842
Train Epoch: 1 	Loss: 0.8630279302597046
Train Epoch: 1 	Loss: 0.8526936769485474
Train Epoch: 2 	Loss: 0.9114435315132141
Train Epoch: 2 	Loss: 0.8351857662200928
Train Epoch: 2 	Loss: 0.757000744342804
Train Epoch: 3 	Loss: 0.847338080406189
Train Epoch: 3 	Loss: 0.7424466013908386
Train Epoch: 3 	Loss: 0.7583439946174622
Train Epoch: 4 	Loss: 0.8391985893249512
Train Epoch: 4 	Loss: 0.6897891163825989
Train Epoch: 4 	Loss: 0.7156673669815063
Train Epoch: 5 	Loss: 0.7029874324798584
Train Epoch: 5 	Loss: 0.7211431264877319
Train Epoch: 5 	Loss: 0.5470129251480103
Train Epoch: 6 	Loss: 0.7174286246299744
Train Epoch: 6 	Loss: 0.6724042296409607
Train Epoch: 6 	Loss: 0.6027325987815857
Train Epoch: 7 	Loss: 0.6024956703186035
Train Epoch: 7 	Loss: 0.6149489879608154
Train Epoch: 7 	Loss: 0.6186243295669556
Train Epoch: 8 	Lo